In [125]:
import os
import cv2
import shutil
import numpy as np
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg19 import decode_predictions

In [126]:
model = VGG16(weights='imagenet')

In [127]:
photo_path='lowre'
new_path='BestPhotos'
if not os.path.exists(new_path):
    os.makedirs(new_path)

In [128]:
def load_and_preprocess_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        images.append((img_path, img_array))
    return images


In [129]:
images=load_and_preprocess_images_from_folder(photo_path)

In [130]:
sorted_images = []
for img_path, x in images:
    predictions = model.predict(x)
    decoded_predictions = decode_predictions(predictions)
    top_prediction = decoded_predictions[0][0]
    sorted_images.append((img_path, top_prediction[2]))

sorted_images.sort(key=lambda x: x[1], reverse=True)
top_3_images = sorted_images[:3]

1/1 [==============================] - 0s 278ms/step


In [131]:
for img_path, _ in top_3_images:
    img_name = os.path.basename(img_path)
    hedef_path = os.path.join(new_path, img_name)
    shutil.copy(img_path, new_path)

#model için uygun formata dönüştürme
for i, x in enumerate(images):
    predictions = model.predict(x)
    
    decoded_predictions = decode_predictions(predictions)
    top_prediction = decoded_predictions[0][0]
    
    # En iyi fotoğrafı güncelleme
    if top_prediction[2] > best_score:
        best_score = top_prediction[2]
        best_image_index = i

if best_image_index is not None:
    best_image = images[best_image_index]
    original_filename = os.listdir(photo_path)[best_image_index]  # Dizin içindeki dosyaların sırasını korumak için
    output_path = os.path.join(photo_path, 'best_image_' + original_filename)
    cv2.imwrite(output_path, best_image)
    cv2.imwrite('best.jpeg', best_image)
    print(f'En iyi fotoğraf kaydedildi: {output_path}')
else:
    print('Hiçbir uygun fotoğraf bulunamadı.')